# 02MIAR Actividad evaluada — ejercicio

❗️TODO: Agregar nombres

Grupo 5 de Matematicas:

- Julio Emanuel Suriano Bryk

Para hacer:

- Completar TODO
- Cambiar comentarios al espaniol


## Setup del proyecto


In [468]:
# data analysis libs
import numpy as np
import pandas as pd

# utils libs
import copy
import random
from functools import reduce
from timeit import timeit

# graphical libs
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# typehint libs
from typing import Callable
from numpy.typing import NDArray

import warnings

# Avoid annoying warning for deprecations ...
warnings.simplefilter(action="ignore", category=FutureWarning)

# make plots with visible on export
pio.renderers.default = "notebook_connected"

In [469]:
# Helpers functions
def get_square_matrix(n: int, n_range: tuple[int, int] = (-10, 10)) -> list[list]:
    """generate random matrix of n size

    Args:
        n (int): amount of rows/columns
        n_range (tuple[int, int], optional): range of numbers for each item in the matrix. Defaults to (0, 100).

    Returns:
        list[list]: generated matrix
    """
    return [[random.randint(*n_range) for _ in range(n)] for _ in range(n)]


def matrix_to_str(matrix: list[list]) -> str:
    """Return a string with the representation of the matrix

    Args:
        matrix (list[list]): matrix that you want to display

    Returns:
        str: representation of matrix
    """
    digits = max(len(f"{num:.2f}") for row in matrix for num in row)
    res = map(lambda row: " ".join([f"{n:{digits}.{2}f}" for n in row]), matrix)
    return "\n".join(res)


dimensions = [1, 2, 3, 4]

for n in dimensions:
    matrix = get_square_matrix(n)
    print(f"Matrix {n}x{n}: ")
    print(matrix_to_str(matrix))
    print("-----")

Matrix 1x1: 
-5.00
-----
Matrix 2x2: 
 0.00 -1.00
10.00 -3.00
-----
Matrix 3x3: 
 -4.00   8.00 -10.00
  0.00   8.00  -6.00
  9.00  -9.00   0.00
-----
Matrix 4x4: 
 -4.00   1.00  -3.00 -10.00
 -3.00   2.00   6.00 -10.00
  7.00   2.00  -8.00 -10.00
 10.00   1.00   8.00  -8.00
-----


## Ejercicio 1 (2 puntos). Desarrollo de Laplace.


1. Deducir de la definición 4 el determinante en dimensión 0, 1 y 2.


❗️TODO: Completar aca las definiciones


2. A partir de la definición 4, expresar el determinante de una matriz cuadrada recursivamente en función de determinantes la matrices cuadradas de dimensión inferior.

Indicación: para cada $𝑛 ∈ N$, distribuir (por linealidad en las columnas) sobre la descomposición

$$
\det\begin{pmatrix} 𝜆 & w \\ v & A \end{pmatrix} = \det\begin{pmatrix} 𝜆 * 1 + 0 & w \\ 𝜆 * 0 + v & A \end{pmatrix}
$$

de una matriz cuadrada de dimensión $𝑛 + 1$, siendo $𝑛 ∈ N$ y

- 𝜆 un coeficiente real,
- 𝑣 un vector de dimensión 𝑛 (una columna de 𝑛 coeficientes reales),
- 𝜔 un covector de la misma dimensión (una fila de 𝑛 coeficientes),
- 𝐴 una matriz cuadrada de la misma dimensión (con 𝑛2 coeficientes),

luego proceder del mismo modo

- con los demás coeficientes de esa primera columna,
- con cada columna.


❗️TODO: completar esta seccion


3. Implementar en Python la definición así obtenida.


In [470]:
def laplace_determinant(matrix: list[list]) -> float:
    """get determinant of a matrix using Laplace recursive method

    Args:
        matrix (list[list]): squared matrix to obtain the determinant from

    Raises:
        Exception: in case the matrix is not squared

    Returns:
        float: value of the determinant
    """
    if len(matrix) == 1:
        return matrix[0]

    if len(matrix) != len(matrix[0]):
        raise Exception("Matriz no es cuadrada...")

    if len(matrix) == 2:
        a, b = matrix[0]
        c, d = matrix[1]
        return a * d - b * c

    res = 0
    for i, n in enumerate(matrix[0]):
        sub_matrix = [[el for j, el in enumerate(row) if j != i] for row in matrix[1:]]
        res += n * (-1) ** i * laplace_determinant(sub_matrix)

    return res


dimensions = [1, 2, 3, 4]

for n in dimensions:
    matrix = get_square_matrix(n)
    print(f"Matrix {n}x{n}: ")
    print(matrix_to_str(matrix))
    print("Determinante:", laplace_determinant(matrix))
    print("-----")

Matrix 1x1: 
8.00
Determinante: [8]
-----
Matrix 2x2: 
 5.00 -6.00
 2.00 -5.00
Determinante: -13
-----
Matrix 3x3: 
 6.00 -5.00  9.00
 4.00 -6.00  9.00
 8.00 -8.00 10.00
Determinante: 56
-----
Matrix 4x4: 
 -1.00   4.00   1.00   1.00
 -7.00  -6.00  10.00  -3.00
  3.00  -7.00  -7.00 -10.00
  3.00  -3.00  -3.00  -5.00
Determinante: 791
-----


## Ejercicio 2 (2 puntos). Eliminación de Gauss–Jordan


1. Deducir de la definición 4 el efecto que tiene en el determinante de una matriz sumar a una de sus columnas una combinación lineal de las demás.


❗️ TODO: Completar


2. A partir de la definición 4, proponer una estrategia para triangularizar una matriz sin cambiar su determinante e implementar en Python una definición alternativa del determinante.

Indicación: descomponer similarmente al ejercicio anterior.


❗️ TODO: Completar


3. Implementar en Python la definición así obtenida.


In [471]:
def gaussian_elimination(initial_matrix: list[list]) -> tuple[list[list], float]:
    """perform the gaussian elimination method for a given matrix

    Args:
        initial_matrix (list[list]): matrix to apply gaussian elimination

    Returns:
        tuple[list[list], float]: triangular matrix along with its determinant
    """
    matrix = copy.deepcopy(initial_matrix)
    n = len(matrix)

    # keep track of row swaps to calculate final determinant
    sign = 1

    for i in range(n - 1):
        # find the proper row pivot to avoid zero division and rounded errors
        max_row = max(range(i, n), key=lambda j: abs(matrix[j][i]))
        if max_row != i:
            matrix[i], matrix[max_row] = matrix[max_row], matrix[i]
            sign *= -1

        # apply rows operations in order to ensure triangularly of the matrix
        for j in range(i + 1, n):
            factor = matrix[j][i] / matrix[i][i]
            for k in range(i, n):
                matrix[j][k] -= matrix[i][k] * factor

    # calculate determinant of a matrix using only the diagonal of it
    diagonal_value = reduce(lambda acc, i: acc * matrix[i][i], range(n), 1)
    determinant = diagonal_value * sign

    return (matrix, determinant)


matrix = get_square_matrix(3)

print("Matriz original:")
print(matrix_to_str(matrix))
print("Determinante:", laplace_determinant(matrix))

print("----")

(triangular_matrix, triangular_determinant) = gaussian_elimination(matrix)
print("Matriz triangular:")
print(matrix_to_str(triangular_matrix))
print("Determinante:", f"{triangular_determinant:.2f}")

Matriz original:
-9.00  3.00 -8.00
-5.00  1.00 10.00
-2.00  4.00 -8.00
Determinante: 396
----
Matriz triangular:
-9.00  3.00 -8.00
 0.00  3.33 -6.22
 0.00  0.00 13.20
Determinante: 396.00


### Ejercicio 3 (2 puntos). Comparación.


1. Obtener la complejidad computacional de cada una de estas dos implementaciones.


Complejidad computacional para el calculo del determinante:

- Definicion con Laplace: $O(n!)$ por su propiedad recursiva
- Definicion con Gauss: $O(n^3)$ son 3 `for` anidados


2. Generar matrices aleatoriamente en dimensión $𝑛 ∈ { 2, 3, · · · , 9, 10 }$ y comparar el tiempo de ejecución de cada una de estas dos implementaciones con la función `numpy.linalg.det` (la función determinante de la extensión numérica de Python al álgebra lineal).

Indicación: se puede utilizar la función `numpy.random.rand` para generar los coeficientes aleatorios de sus matrices.


In [472]:
data = []

input_sizes = [np.random.rand(n, n).tolist() for n in range(2, 11)]

for matrix in input_sizes:
    n = len(matrix)
    recur_time = timeit(lambda: laplace_determinant(matrix), number=1)
    gauss_time = timeit(lambda: gaussian_elimination(matrix), number=1)
    numpy_time = timeit(lambda: np.linalg.det(matrix), number=1)

    data.extend(
        [
            ["Recursive", n, recur_time],
            ["Gauss", n, gauss_time],
            ["Numpy", n, numpy_time],
        ]
    )


df = pd.DataFrame(data, columns=["Method", "n", "Time"])

fig = px.line(
    df,
    x="n",
    y="Time",
    color="Method",
    markers=True,
    title="Time to calculate matrix determinant",
)

fig.show()

## Ejercicio 4 (4 puntos).

Con el propósito de aproximar un mínimo local de una función real de varias variables reales, el método de descenso de gradiente consiste en iterar una marcha (positivamente) proporcional al (opuesto del) gradiente desde un valor inicial, con la intuición de ‘seguir el agua’ hasta dar con el valle.


In [473]:
# helpers to draw functions


def draw_function_2d_with_gradient(
    f: Callable,
    graph_range: tuple[float, float],
    descent_history: NDArray,
    title: str,
):
    """Create plot using plotly to represent a gradient descent result of function

    Args:
        f (Callable): function to render
        graph_range (tuple[float, float]): range to draw the function
        descent_history (NDArray): history of the gradient descent algorithm
        title (str): title for the graph
    """
    # Vectorize function in order to be possible to support numpy array operations
    f_vector = np.vectorize(f)

    x = np.linspace(*graph_range, 100)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=x, y=f_vector(x), mode="lines", name="Function"))

    descent_df = pd.DataFrame(
        [(*p, f_vector(*p)) for p in descent_history], columns=["x", "y"]
    )

    try:
        fig.add_trace(
            go.Scatter(
                x=descent_df["x"],
                y=descent_df["y"],
                mode="markers",
                name="Gradient",
            )
        )
    except OverflowError:
        print("Cannot draw gradient history due to Overflow Error")

    fig.update_layout(title=title)

    fig.show()


def draw_function_3d_with_gradient(
    f: Callable,
    graph_range: tuple[float, float],
    descent_history: NDArray,
    title: str,
):
    """Create plot using plotly to represent a gradient descent result of function

    Args:
        f (Callable): function to render
        graph_range (tuple[float, float]): range to draw the function
        descent_history (NDArray): history of the gradient descent algorithm
        title (str): title for the graph
    """
    # Vectorize function in order to be possible to support numpy array operations
    f_vector = np.vectorize(f)

    # Create a grid of x and y values
    x, y = np.meshgrid(np.linspace(*graph_range, 10), np.linspace(*graph_range, 10))
    z = f_vector(x, y)

    # Create the plot
    fig = go.Figure(data=[go.Surface(z=z, x=x, y=y, opacity=0.7)])

    # convert history to pandas df
    descent_df = pd.DataFrame(
        [(*p, f_vector(*p)) for p in descent_history], columns=["x", "y", "z"]
    )

    # mark start and end point of the gradient history
    points_df = pd.concat([descent_df.iloc[[0]], descent_df.iloc[[-1]]])
    fig.add_traces(
        go.Scatter3d(
            x=points_df["x"],
            y=points_df["y"],
            z=points_df["z"],
            mode="markers+text",
            text=["Start", "End"],
        )
    )

    # add gradient line
    fig.add_traces(
        px.line_3d(
            descent_df, x="x", y="y", z="z", color_discrete_sequence=["white"]
        ).data
    )

    fig.update_layout(title=title)

    # Show the plot
    fig.show()

1. Implementar en Python un algoritmo de descenso de gradiente (con un máximo de $m = 10**5$ iteraciones) a partir de los siguientes argumentos tomados en ese orden:

- la función $f$ cuyo mínimo local se propone aproximar,
- el valor inicial $x$ desde el que empieza la marcha,
- la razón geométrica o coeficiente de proporcionalidad $y$,
- el parámetro de tolerancia $z$ para finalizar cuando el gradiente de la función $f$ caiga dentro de esa tolerancia.

Indicación: empezar por implementar el gradiente `grad(f)` de la función $f$


In [474]:
# step size
grad_step_size = 0.0001


def get_gradient_using_step(f: Callable, x: NDArray) -> NDArray:
    """calculate gradient for the function at a given point by using the method of

    Args:
        f (Callable): _description_
        x (NDArray): _description_

    Returns:
        NDArray: _description_
    """
    grad = np.zeros_like(x)

    for i in range(len(x)):
        point_with_step = x.copy()
        point_with_step[i] = point_with_step[i] + grad_step_size
        grad[i] = (f(*point_with_step) - f(*x)) / grad_step_size

    return grad


def grad(
    f: Callable,
    x: NDArray,
    y=0.01,
    m=10**5,
    z=1e-6,
) -> list[NDArray]:
    """get minimum point of a function using method of gradient descent with step

    Args:
        f (Callable): function to find the minimum point
        x (NDArray): start point to start looking for the minimum point
        y (float, optional): learning rate to move the point on each iteration. Defaults to 0.01.
        m (int, optional): numbers of max iterations to run. Defaults to 10**5.
        z (float, optional): tolerance value to finish the algorithm in case we found point of convergence. Defaults to 1e-6.

    Returns:
        list[NDArray]: history of the points inside the algorithm of gradient descent
    """
    point = x
    history = np.array([point])

    for _ in range(m):
        # get gradient from the derivate function
        grad = get_gradient_using_step(f, point)

        # Check for point convergence
        if np.linalg.norm(grad) * 2 < z:
            break

        # get new point for the next iteration
        point -= grad * y

        # save it in history
        history = np.concatenate((history, [point]), axis=0)

    return history

2. Calcular formalmente ${ 𝑡 ∈ R. 𝑓 ′(𝑡) = 0 }$ para $f(t) = 3t^4 + 4t^3 − 12t^2 + 7$


$$
  f'(t) = 12 t^3 + 12 t^2 - 24 t \\
  f'(t) = 12 t (t^2 + t - 2)
$$

Para que $f'(t) = 0$, entonces los puntos criticos son:

$$
  t = -2\\
  t = 0 \\
  t = 1 \\
$$


3. Con una tolerancia $z = 10**-12$ y un valor inicial de $x = 3$ aplicar su algoritmo con razón $y = 10**-1, 10**-2, 10**-3$ luego hacer lo mismo con $x = 0$. Interpretar el resultado.


❗TODO: agregar interpretaciones


In [475]:
def f(x: float) -> float:
    return 3 * (x**4) + 4 * (x**3) - 12 * x**2 + 7


descent_history = grad(f, x=[3], y=0.001)
draw_function_2d_with_gradient(
    f,
    (-3, 3),
    descent_history,
    title=f"x = 3 y=10^-3 encontro punto minimo (local) en {descent_history[-1]}",
)

descent_history = grad(f, x=[3], y=0.01)
draw_function_2d_with_gradient(
    f,
    (-3, 3),
    descent_history,
    title=f"x = 3 y=10^-2 encontro punto minimo (global) en {descent_history[-1]}",
)

descent_history = grad(f, x=[3], y=0.1)
draw_function_2d_with_gradient(
    f,
    (-3, 3),
    descent_history,
    title="x = 3 y=10^-1 no encontro un punto minimo valido por el alto valor de learning rate",
)

descent_history = grad(f, x=[0], y=0.001)
draw_function_2d_with_gradient(
    f,
    (-3, 3),
    descent_history,
    title="x=0 y=10^-1 Es un punto de silla de f, por lo que no convergio a ningun punto minimo",
)

4. Repetir estos dos últimos apartados con $f: (s,t) → s^2 + 3st + t^3 + 1$ y los valores iniciales $x = [-1,1], [0,0]$


Derivadas parciales:

$$
  f'(s,t)_s = 2s + 3t \\
  f'(s,t)_t = 3s + 3t ^2 \\
$$

Para que $f'(t) = 0$, entonces los puntos criticos son:

$$
  (s_1=0,t_1=0) \\
  (s_2= 9/4, t_2= 3/2)
$$


❗TODO: agregar interpretaciones

Ema: solucionar grafico


In [476]:
def f(s: float, t: float) -> float:
    return s**2 + 3 * s * t + t**3 + 1


descent_history = grad(f, x=[-0, 1], y=0.001)
title = f"x = [-1,1] y=10^-3 encontro punto minimo (local) en {descent_history[-1]}"
draw_function_3d_with_gradient(f, (-5, 5), descent_history, title=title)

# same results here ...
# descent_history = grad(f, x=[-1, 1], y=0.01)
# title = f"x = [-1,1] y=10^-2 encontro punto minimo (local) en {descent_history[-1]}"
# draw_function_3d_with_gradient(f, (-5, 5), descent_history, title=title)

# descent_history = grad(f, x=[-1, 1], y=0.1)
# title = f"x = [-1,1] y=10^-1 encontro punto minimo (local) en {descent_history[-1]}"
# draw_function_3d_with_gradient(f, (-5, 5), descent_history, title=title)

# descent_history = grad(f, x=[0, 0], y=0.1)
# title = "x=[0,0] y=10^-1 Es un punto de silla de f, por lo que no convergio a ningun punto minimo"
# draw_function_3d_with_gradient(f, (-10, 10), descent_history, title=title)